In [57]:
import pandas as pd
from multiprocessing import Process, Queue

In [58]:
from functools import partial
import multiprocessing
from multiprocessing import Pool

In [59]:
pd.set_option('display.max_columns', None)

In [60]:
def markDataFrameByChunks(df, num, col):
    ordNums = np.array(sorted(df[col].unique(),reverse=False))
    ordRange = range(0, len(ordNums))
    chunksDF = pd.DataFrame({'colName': ordNums, 'theirNum': ordRange}, index = ordRange)
    chunksDF['chunk'] = pd.cut(chunksDF['theirNum'], num)
    return chunksDF

In [62]:
# use relevant data to load
prices_beta = pd.read_csv("../data/sample/prices_beta.csv")

In [63]:
prices_beta.head()

,ticker,datetime,priceBtc,priceUsd,volume
0,NGC,2018-03-25T02:00:00Z,0.000081,0.686255,172382.0
1,PRIX,2018-04-02T08:00:00Z,0.000204,1.438895,5437.0
2,AID,2018-04-25T12:00:00Z,0.000019,0.178030,234603.0
3,VIA,2018-04-10T22:00:00Z,0.000228,1.557606,9924460.0
4,STU,2018-03-31T04:00:00Z,0.000003,0.022287,431363.0


In [64]:
# assign chunk numbers
prices_betaWithChunks = markDataFrameByChunks(prices_beta,3,'ticker')
tmpDF = prices_beta.merge(prices_betaWithChunks, left_on = 'ticker', right_on = 'colName', how = 'left')
tmpDF.drop(['colName','theirNum'], axis = 1, inplace = True)
print tmpDF.shape

(100, 6)


In [66]:
# check chunk numbers
dfByChunk = tmpDF.groupby('chunk')
for name, group in dfByChunk:
    print group.shape[0]

30
29
30


In [69]:
def parallel_template(df, param1):
    print df, param1[1].shape
    return {'key': 1, 'value':2}

In [70]:
%%time
#dfByChunk = dfByChunk.groupby('chunk')
dfByChunk = prices_betaWithChunks.groupby('chunk')
func = partial(parallel_template, 1)
if __name__ == '__main__':
    p = Pool(processes=3)
    print 'defined workers'
    result = (p.map(func, dfByChunk))
    p.close()
    try:
        p.terminate()
    except WindowsError:
        pass
    p.join()
    #dfByChunk = None
    print 'closed pool'
p.terminate()

1 (30, 3)
1 (29, 3)
1 (30, 3)
defined workers
closed pool
CPU times: user 12.5 ms, sys: 21.4 ms, total: 33.8 ms
Wall time: 130 ms


In [77]:
%%time
# unpack list with outputs
unpackedDF = pd.DataFrame()
for res in result:
    print res
    tmpDF = pd.DataFrame(res, index = range(0,1))
    unpackedDF = pd.concat([unpackedDF, tmpDF])
    #for subRes in res:
    #    tmpDF = pd.DataFrame(subRes)
    #    datesDF = pd.concat([datesDF, tmpDF])

{'key': 1, 'value': 2}
{'key': 1, 'value': 2}
{'key': 1, 'value': 2}
CPU times: user 6.75 ms, sys: 3.72 ms, total: 10.5 ms
Wall time: 9.93 ms
